<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


 ## Lab 2: Data wrangling 


Estimated time needed: **60** minutes


In this lab, we will perform some Exploratory Data Analysis (EDA) to find some patterns in the data and determine what would be the label for training supervised models. 

In the data set, there are several different cases where the booster did not land successfully. Sometimes a landing was attempted but failed due to an accident; for example, <code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully landed on  a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed on a drone ship. 

In this lab we will mainly convert those outcomes into Training Labels with `1` means the booster successfully landed `0` means it was unsuccessful.


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


## Objectives
Perform exploratory  Data Analysis and determine Training Labels 

- Exploratory Data Analysis
- Determine Training Labels 


----


Install the below libraries


In [2]:
!pip install pandas
!pip install numpy

## Import Libraries and Define Auxiliary Functions


We will import the following libraries.


In [3]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np

### Data Analysis 


Load Space X dataset, from last section.


In [4]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Identificar y calcular el porcentaje de valores faltantes en cada atributo


In [14]:
df.isnull().sum()/df.count()*100

FlightNumber       0.000
Date               0.000
BoosterVersion     0.000
PayloadMass        0.000
Orbit              0.000
LaunchSite         0.000
Outcome            0.000
Flights            0.000
GridFins           0.000
Reused             0.000
Legs               0.000
LandingPad        40.625
Block              0.000
ReusedCount        0.000
Serial             0.000
Longitude          0.000
Latitude           0.000
landing_class      0.000
dtype: float64

Identify which columns are numerical and categorical:


In [15]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
landing_class       int64
dtype: object

### TASK 1: Calcular el número de lanzamientos en cada sitio

Los datos contienen varias instalaciones de lanzamiento de Space X: <a href='https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites'>Complejo de lanzamiento espacial de Cabo Cañaveral</a> 40 <b>VAFB SLC 4E </b>, Complejo de lanzamiento espacial de la base de la fuerza aérea Vandenberg 4E <b>(SLC-4E)</b>, Complejo de lanzamiento espacial del Centro Espacial Kennedy 39A <b>KSC LC 39A </b>. La ubicación de cada lanzamiento se coloca en la columna <code>LaunchSite</code>


A continuación, veamos la cantidad de lanzamientos de cada sitio.

Utilice el método value_counts() en la columna LaunchSite para determinar la cantidad de lanzamientos en cada sitio:


In [7]:
# Apply value_counts() on column LaunchSite
df['LaunchSite'].value_counts()

LaunchSite
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64

Cada lanzamiento apunta a una órbita dedicada, y estos son algunos tipos de órbitas comunes:


* <b>LEO</b>: La órbita terrestre baja (LEO) es una órbita centrada en la Tierra con una altitud de 2000 km (1200 mi) o menos (aproximadamente un tercio del radio de la Tierra),[1] o con al menos 11,25 períodos por día (un período orbital de 128 minutos o menos) y una excentricidad menor a 0,25.[2] La mayoría de los objetos creados por el hombre en el espacio exterior están en LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a>.

* <b>VLEO</b>: Las órbitas terrestres muy bajas (VLEO) se pueden definir como las órbitas con una altitud media inferior a 450 km. Operar en estas órbitas puede proporcionar una serie de beneficios a las naves espaciales de observación de la Tierra, ya que la nave espacial opera más cerca del punto de observación<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>.

* <b>GTO</b> Una órbita geoestacionaria es una órbita terrestre alta que permite a los satélites adaptarse a la rotación de la Tierra. Ubicada a 22,236 millas (35,786 kilómetros) sobre el ecuador de la Tierra, esta posición es un punto valioso para monitorear el clima, las comunicaciones y la vigilancia. Debido a que el satélite orbita a la misma velocidad que gira la Tierra, el satélite parece permanecer en su lugar a lo largo de una única longitud, aunque puede desplazarse de norte a sur”, escribió la NASA en su sitio web Earth Observatory <a href="https://www.space.com/29222-geosynchronous-orbit.html" >[3] </a>.

* <b>SSO (o SO)</b>: Es una órbita heliosincrónica, también llamada órbita heliosincrónica, es una órbita casi polar alrededor de un planeta, en la que el satélite pasa sobre cualquier punto dado de la superficie del planeta a la misma hora solar media local <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4] <a>.

* <b>ES-L1</b>: En los puntos de Lagrange, las fuerzas gravitacionales de los dos cuerpos grandes se cancelan de tal manera que un objeto pequeño colocado en órbita allí está en equilibrio en relación con el centro de masa de los cuerpos grandes. L1 es uno de esos puntos entre el Sol y la Tierra <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a> .

* <b>HEO</b> Una órbita altamente elíptica es una órbita elíptica con alta excentricidad, generalmente se refiere a una órbita alrededor de la Tierra <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6]</a>.

* <b> ISS </b> Una estación espacial modular (satélite artificial habitable) en órbita terrestre baja. Se trata de un proyecto colaborativo multinacional entre cinco agencias espaciales participantes: NASA (Estados Unidos), Roscosmos (Rusia), JAXA (Japón), ESA (Europa) y CSA (Canadá). <a href="https://en.wikipedia.org/wiki/International_Space_Station"> [7] </a>

* <b> MEO </b> Órbitas geocéntricas que varían en altitud desde los 2000 km (1200 mi) hasta justo por debajo de la órbita geosincrónica a 35 786 kilómetros (22 236 mi). También conocida como órbita circular intermedia. Estos se encuentran "más comúnmente a 20.200 kilómetros (12.600 mi), o 20.650 kilómetros (12.830 mi), con un período orbital de 12 horas <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8] </a>

* <b> HEO </b> Órbitas geocéntricas por encima de la altitud de la órbita geosincrónica (35.786 km o 22.236 mi) <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [9] </a>

* <b> GEO </b> Es una órbita geosincrónica circular a 35.786 kilómetros (22.236 millas) sobre el ecuador de la Tierra y que sigue la dirección de la rotación de la Tierra <a href="https://en.wikipedia.org/wiki/Geostationary_orbit"> [10] </a>

* <b> PO </b> Es un tipo de satélite en el que un satélite pasa por encima o casi por encima de ambos polos del cuerpo que orbita (normalmente un planeta como la Tierra <a href="https://en.wikipedia.org/wiki/Polar_orbit"> [11] </a>

algunos se muestran en el siguiente gráfico:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)


### TAREA 2: Calcular el número y ocurrencia de cada órbita


Utilice el método <code>.value_counts()</code> para determinar el número y la ocurrencia de cada órbita en la columna <code>Orbit</code>


In [8]:
# Apply value_counts on Orbit column
ocurrencias = df['Orbit'].value_counts()
ocurrencias

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: count, dtype: int64

### TAREA 3: Calcular el número y la ocurrencia del resultado de la misión de las órbitas.


Utilice el método <code>.value_counts()</code> en la columna <code>Outcome</code> para determinar el número de <code>landing_outcomes</code>. Luego asígnelo a una variable landing_outcomes.


In [9]:
# landing_outcomes = values on Outcome column
landing_outcomes = df['Outcome'].value_counts()
landing_outcomes

Outcome
True ASDS      41
None None      19
True RTLS      14
False ASDS      6
True Ocean      5
False Ocean     2
None ASDS       2
False RTLS      1
Name: count, dtype: int64

<code>True Ocean</code> significa que el resultado de la misión aterrizó con éxito en una región específica del océano, mientras que <code>False Ocean</code> significa que el resultado de la misión aterrizó sin éxito en una región específica del océano. <code>True RTLS</code> significa que el resultado de la misión aterrizó con éxito en una plataforma de tierra. <code>False RTLS</code> significa que el resultado de la misión aterrizó sin éxito en una plataforma de tierra. <code>True ASDS</code> significa que el resultado de la misión aterrizó con éxito en un barco no tripulado. <code>False ASDS</code> significa que el resultado de la misión aterrizó sin éxito en un barco no tripulado. <code>None ASDS</code> y <code>None None</code> representan un aterrizaje fallido.


In [10]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


We create a set of outcomes where the second stage did not land successfully:


In [11]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

### TASK 4: Create a landing outcome label from Outcome column


Utilizando <code>Outcome</code>, crea una lista donde el elemento sea cero si la fila correspondiente en <code>Outcome</code> está en el conjunto <code>bad_outcome</code>; de lo contrario, es uno. Luego asígnalo a la variable <code>landing_class</code>:


In [17]:
df['Class']=df['Outcome'].apply(lambda landing_class:0 if landing_class in bad_outcomes else 1)
df[['Class']].head(8)


,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


Esta variable representará la variable de clasificación que representa el resultado de cada lanzamiento. Si el valor es cero, la primera etapa no aterrizó con éxito; uno significa que la primera etapa aterrizó con éxito

In [19]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,landing_class,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,1,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,1,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,1,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,1,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,1,0


We can use the following line of code to determine  the success rate:


In [20]:
df["Class"].mean()

0.6666666666666666

We can now export it to a CSV for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range.


<code>df.to_csv("dataset_part_2.csv", index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a> is a Data Scientist at IBM and pursuing a Master of Management in Artificial intelligence degree at Queen's University.


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-08-31        | 1.1     | Lakshmi Holla    | Changed Markdown |
| 2020-09-20        | 1.0     | Joseph     | Modified Multiple Areas |
| 2020-11-04        | 1.1.    | Nayef      | updating the input data |
| 2021-05-026       | 1.1.    | Joseph      | updating the input data |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
